In [3]:
import os
from PIL import Image, ImageDraw
import numpy as np

# Create folder structure
base_dir = 'data/recycle'
classes = ['recyclable', 'non_recyclable']

for cls in classes:
    os.makedirs(os.path.join(base_dir, cls), exist_ok=True)

# Function to generate simple synthetic images
def generate_image(cls, index):
    img = Image.new('RGB', (64, 64), color='white')
    draw = ImageDraw.Draw(img)
    if cls == 'recyclable':
        draw.rectangle([16, 16, 48, 48], fill='green')  # green square
    else:
        draw.ellipse([16, 16, 48, 48], fill='red')      # red circle
    img.save(os.path.join(base_dir, cls, f'{cls}_{index}.png'))

# Generate 10 images per class
for cls in classes:
    for i in range(10):
        generate_image(cls, i)

print("Synthetic dataset created successfully!")


Synthetic dataset created successfully!


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = train_gen.flow_from_directory(
    'data/recycle',
    target_size=(64, 64),
    class_mode='binary',
    subset='training'
)

val_data = train_gen.flow_from_directory(
    'data/recycle',
    target_size=(64, 64),
    class_mode='binary',
    subset='validation'
)


Found 16 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [5]:
from tensorflow.keras import layers, models

# Build a simple CNN
model = models.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(64,64,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(train_data, validation_data=val_data, epochs=5)


Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 0.7421 - val_accuracy: 0.5000 - val_loss: 0.5981
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - accuracy: 0.5000 - loss: 0.5981 - val_accuracy: 1.0000 - val_loss: 0.4680
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - accuracy: 1.0000 - loss: 0.4680 - val_accuracy: 1.0000 - val_loss: 0.3649
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step - accuracy: 1.0000 - loss: 0.3649 - val_accuracy: 1.0000 - val_loss: 0.2663
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 1.0000 - loss: 0.2663 - val_accuracy: 1.0000 - val_loss: 0.1831


In [6]:
import tensorflow as tf

# Convert the trained model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open("recycle_model.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpqnk6mt0i'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132035566748880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132035572957008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132035566747920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132035566749456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132035566749072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132035566750608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132035572956816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132035566749648: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [7]:
interpreter = tf.lite.Interpreter(model_path="recycle_model.tflite")
interpreter.allocate_tensors()

# Get input/output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare a single test image
img = np.array(val_data[0][0][0], dtype=np.float32).reshape(1,64,64,3)
interpreter.set_tensor(input_details[0]['index'], img)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
print("Predicted probability:", output[0][0])


Predicted probability: 0.109311804


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
